In [4]:
# load src/
import sys
import random
sys.path.append('../')
import src.dataset

import numpy as np
from tqdm.auto import tqdm, trange

import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
# src.dataset.create_n_splits("../data/data_train.csv")

In [8]:
X_train, X_valid, X_test = src.dataset.load_datasets()

Caching dataset ../data/data_train.csv.0 ...



Caching dataset ../data/data_train.csv.1 ...



Caching dataset ../data/data_train.csv.2 ...



Caching dataset ../data/data_train.csv.3 ...



Caching dataset ../data/sampleSubmission.csv ...



Caching dataset ../data/data_train.csv.4 ...


In [3]:
# For final submission:
#import scipy
#X_train = X_train + X_valid
#mean = X_train[X_train.nonzero()].mean()

In [4]:
!nvidia-smi

Wed Jun 17 11:47:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    On   | 00000000:01:00.0 Off |                  N/A |
|  0%   36C    P8    16W / 230W |     42MiB /  8117MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [9]:
X_train.count_nonzero(), X_valid.count_nonzero(), X_test.count_nonzero()
valid_indices = list(set(zip(X_train.nonzero()[0], X_train.nonzero()[1])))
# valid_per_line = [[] for _ in range(10000)]
# for i, j in valid_indices:
#     valid_per_line[i].append((i,j))

In [10]:
# from collections import Counter
# def count_vals(X, valid_indices):
#     counts = {}
#     for i, j in valid_indices:
#         val = X[i, j]
#         counts.setdefault(val, 0)
#         counts[val] += 1
#     return counts
# print(count_vals(X_train, valid_indices))

In [11]:
# import random
# X_unbias = X_train.copy()
# for i, j in valid_indices:
#     val = X_train[i, j]
#     if val >= 3 and random.random() < 0.5:
#         X_unbias[i, j] = 0

In [12]:
# valid_indices_unbias = list(set(zip(X_unbias.nonzero()[0], X_unbias.nonzero()[1])))
# print(count_vals(X_unbias, valid_indices))

In [33]:
def _add_regularization(model):
    l2_reg = None
    for W in model.parameters():
        if l2_reg is None:
            l2_reg = W.norm(2)
        else:
            l2_reg = l2_reg + W.norm(2)
    return l2_reg

In [34]:
class Autoencoder(nn.Module):

    def __init__(self):
        super().__init__()
        self.user_dim = 10000
        self.item_dim = 1000
        self.embed_dim = 200
#         self.user_embed = nn.Embedding(self.user_dim, self.embed_dim)
#         self.item_embed = nn.Embedding(self.item_dim, self.embed_dim)
        self.layer_1 = nn.Linear(1000, 600)
#         self.layer_1a = nn.Linear(300, 300)
        #self.layer_2 = nn.Linear(300, 100)
        
        self.vae = nn.Linear(600, 100)
        
        self.layer_3 = nn.Linear(50, 600)
        
        self.cls_layer = nn.Linear(600, 1000)
    
    def forward(self, data):
#         user_idx, item_idx = torch.split(data, 1, dim=1)
#         print("user",user_idx)
#         print("item",item_idx)
#         user_idx = torch.squeeze(user_idx, dim=-1)
#         item_idx = torch.squeeze(item_idx, dim=-1)

#         user_embedding = self.user_embed(user_idx)
#         item_embedding = self.item_embed(item_idx)
#         assert user_embedding.shape[-1] == item_embedding.shape[-1] == self.embed_dim

        # Input is concatenation
#         net_data = torch.cat([user_embedding, item_embedding], dim=-1)
        net_data = data

        # Feedforward layers
        #net_data = F.relu(self.layer_1(net_data))
        #net_data = F.dropout(net_data)
        
        net_data = F.tanh(self.layer_1(net_data))
        net_data = F.dropout(net_data)
#         net_data = F.relu(self.layer_1a(net_data))
#         net_data = F.dropout(net_data)
        
        #net_data = F.relu(self.layer_2(net_data))
        #net_data = F.dropout(net_data)
        
        vae = self.vae(net_data)
        mus_q, log_sigmas_q = torch.split(vae, 100, dim=-1)
        stds_q = torch.exp(0.5 * log_sigmas_q)
        
        KL = 0.5 * (-log_sigmas_q + torch.exp(log_sigmas_q) + mus_q ** 2 - 1)
        KL = torch.sum(KL, dim=-1)
        KL = torch.mean(KL)
        
        if not self.training:
            sampled_z = mus_q
        else:
            eps = torch.randn(stds_q.shape, dtype=torch.float, device=device)
            sampled_z = mus_q + eps * stds_q
        
        #net_data = F.relu(self.layer_3(net_data))
        #net_data = F.dropout(net_data)
        
        #net_data = F.relu(self.layer_3(sampled_z))
        #net_data = F.dropout(net_data)
        
        net_data = F.tanh(self.layer_3(sampled_z))
        net_data = F.dropout(net_data)
        
        # Predict score for position A_i_j # Andreas: Wouldn't this be for all items A_i for user i?
        y_score = self.cls_layer(net_data)
#         y_score = (4.0 * torch.sigmoid(y_score)) + 1.0
#         return torch.sigmoid(y_score) * 6.0
        return y_score, KL

In [35]:
try:
    del model
except:
    pass
model = Autoencoder().cuda()
# model = Autoencoder()

In [36]:
device = torch.device('cuda')
# device = torch.device('cpu')
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

In [37]:
print(X_train.count_nonzero(), X_valid.count_nonzero())

def get_predictions(A):
    model.eval()
#     model.train()
    A_pred = np.zeros((10000, 1000))
    
    valid_indices = list(set(zip(A.nonzero()[0], A.nonzero()[1])))
    
    for i in range(0, 10000, 64):
        if i % (64 * 10) == 0:
            print(i)
#        X = valid_indices[i:i+64]
        X = X_train[i:i+64]
        X_ = torch.tensor(X.todense(), dtype=torch.float, device=device)
        y_preds, KL = model(X_)
        if USE_CLS:
            y_preds = torch.argmax(torch.softmax(y_preds, dim=-1), dim=-1)
            if i == 0:
                print(y_preds)
        for i_ in range(y_preds.shape[0]):
            for j_ in range(y_preds.shape[1]):
#                 if (i+i_,j) in valid_indices:
                #print(x_ij)
                x_ij = y_preds[i_, j_]
                #print((i+i_), j_)
                A_pred[i+i_, j_] = x_ij
        
    #print(A_pred[0])
    print("Stats:", np.mean(A_pred[A_pred.nonzero()]), np.std(A_pred.nonzero()))
    return A_pred, valid_indices

def compute_loss():
    A_pred, valid_indices = get_predictions(X_valid)
    losses = np.square(X_valid - A_pred)
    losses = [losses[i,j] for (i, j) in valid_indices]
    mean_loss = np.mean(losses)
    return mean_loss

#compute_loss()

925567 251385


In [38]:
train_indices_perline = [[] for i_ in range(10000)]
for i, j in valid_indices:
    train_indices_perline[i].append((i, j))

for line in train_indices_perline:
    assert line
assert len(train_indices_perline) == 10000

In [42]:
loss_avg = []
USE_CLS = False
anneal_alpha = 1 # Start with 1, then select anneal_alpha for which performance decreases and run again
alpha = 0
iters = 300000


for i in range(0, iters):
    # randomly sample 32 points from training matrix
    model.train()
    #X = random.sample(valid_indices, k=32)
    random_lines = random.sample(range(0, 10000), k=16)
    X = X_train[random_lines].todense()
    
#     y_true = X
    #y_true = [[X_train[Xi]] for Xi in X]
    X = torch.tensor(X, dtype=torch.float, device=device)
#     y_true = torch.tensor(y_true, dtype=torch.long if USE_CLS else torch.float, device=device)
    # step!
    y_true = X
    
    #########
#     X = torch.tensor([[-1, -1], [0, 0], [2, 2]], dtype=torch.float)
#     y_true = torch.tensor([[-1.], [0.], [2.]], dtype=torch.float)
    #########
    
    opt.zero_grad()
    y_preds, KL = model(X)
    
    valid_mask = torch.zeros_like(y_preds, device=device)
    for i_ in range(X.shape[0]):
        i_real = random_lines[i_]
        indices = [j for _,j in train_indices_perline[i_real]]
        valid_mask[i_, indices] = 1.0
#     y_preds *= valid_mask
    
    if USE_CLS:
        loss_fn = torch.nn.CrossEntropyLoss(
            weight=torch.tensor((0., 3., 3., 1., 1., 1.), device=device))
#         print(y_preds.shape, y_true.shape)
        loss = loss_fn(y_preds, y_true.reshape(-1,))
    else:
        loss = (y_preds - y_true) ** 2
        loss = loss[valid_mask != 0.]
#         print(y_true[valid_mask != 0.])
#         print(loss)
    
    neg_ELBO = loss + alpha * KL# + 2 * reg_var
    
    # update avg loss
    loss_avg.insert(0, loss.mean().item())
    loss_avg = loss_avg[:500]
    
    if i % 1000 == 0:
        print("It %d Avg loss: %.4f" % (i, np.mean(loss_avg)), "  alpha is: ", alpha)
    if i and i % 50000 == 0:
        mean_loss = compute_loss()
        print("Mean loss: %.4f" % mean_loss, "  alpha is: ", alpha)
    
#     loss += _add_regularization(model) * 0.1
    #loss.mean().backward()
    neg_ELBO.mean().backward()
    opt.step()
    
    # update alpha
    alpha = (i/iters)*anneal_alpha


It 0 Avg loss: 1.2206   alpha is:  0
It 1000 Avg loss: 1.2499   alpha is:  0.00333


KeyboardInterrupt: 

In [ ]:
# 1. Don't use the classifier, it's not good
# 2. With bs=32, clsweights 22111 adam and 100 neurons it works
# Trained on 1e-4 to 100k, 1.035 valid. loss
# Trained on 1e-5 to 150k, 1.025 valid. loss
# Trained on 1e-5 to 200k, 1.025 valid. loss
# 3. With clsweights 33111 it is the same up to 100k
# 4. With 200 and 300 neurons (150 embedding) the same up to 100k
# 5. With bottleneck of 50 neurons the same up to 100k
## SCORE: 1.05, not good
# Avg loss @100k: 0.95, @200k: 0.92

# With everything, avg loss @300k: 0.87, but worse than before
# Again just 4/5:
# 0.98 and 1.04

In [21]:
mean_loss = compute_loss()
print(mean_loss)

0
640
1280
1920
2560
3200
3840
4480
5120
5760
6400
7040
7680
8320
8960
9600
Stats: 3.5463887094020845 3037.7691958903
16.06283715060512


In [ ]:
# Submission

In [ ]:
import pandas as pd

def export_and_save(target, preds):
    target_rows, target_cols = target.nonzero()
    ids = [f"r{row+1}_c{col+1}" for row, col in zip(target_rows, target_cols)]
    scores = [np.round(preds[row, col]) for row, col in zip(target_rows, target_cols)]
    df = pd.DataFrame({"Id": ids, "Prediction": scores})
    df.to_csv("preds.csv", index=False, float_format='%.4f')

export_and_save(X_test, get_predictions(X_test)[0])